In [1]:
import numpy as np
import pandas as pd
from data import Vertebral_column, Vertebral_column_KF
from data import seismic_bumps
from data import churn
from data import indian_liver_patient
from data import spect_heart, Co_Author
import svm
import trainning_of_adaboost as toa
from sklearn.ensemble import AdaBoostClassifier
import adaboost_svm
from report import report
#from sklearn.metrics import f1_score
from sklearn.metrics  import classification_report, precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, f1_score, precision_score
import math
from sklearn.ensemble import AdaBoostClassifier
from datetime import datetime
import csv
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

In [2]:
def compute_metrics(y_test,y_pred):
    cm_WSVM = confusion_matrix(y_test, y_pred)
    se = cm_WSVM[1,1]/(cm_WSVM[1,0]+cm_WSVM[1,1])
    sp = cm_WSVM[0,0]/(cm_WSVM[0,0]+cm_WSVM[0,1])
    gmean = math.sqrt(se*sp)
    f1s = f1_score(y_test,y_pred)
    acc = accuracy_score(y_test,y_pred)
    pre = precision_score(y_test,y_pred)
    auc = roc_auc_score(y_test, y_pred)

    return sp, se, gmean, f1s, acc, pre, auc, cm_WSVM

In [3]:
def ada_decisiontree(X_train, y_train,X_test):
    clf = AdaBoostClassifier(n_estimators=100)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return y_pred

In [ ]:
from sklearn.svm import SVC
def svm(X_train, y_train,X_test):
    clf = SVC(probability=True, kernel='linear')
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return y_pred

In [4]:
def ada_wsvm(M, C, theta, X_train, y_train,X_test):
    w, b, a = toa.fit(X_train, y_train, M, C, instance_categorization=True, proposed_preprocessing = False,proposed_alpha = False, test_something = False, theta=theta)
    y_pred = toa.predict(X_test, w, b, a, M)
    return y_pred

In [5]:
def imada1_wsvm(M, C, theta, X_train, y_train,X_test):
    w, b, a = toa.fit(X_train, y_train, M, C, instance_categorization=True, proposed_preprocessing = True, proposed_alpha = False, test_something = True, theta=theta)
    y_pred = toa.predict(X_test, w, b, a, M)
    return y_pred

In [6]:
def imada2_wsvm(M, C, X_train, y_train,X_test):
    w, b, a = toa.fit(X_train, y_train, M, C, instance_categorization = True,proposed_preprocessing = False,proposed_alpha=True,test_something = True)
    y_pred = toa.predict(X_test, w, b, a, M)
    return y_pred

In [7]:
def imada_12_wsvm(M, C, X_train, y_train,X_test):
    w, b, a = toa.fit(X_train, y_train, M, C, instance_categorization = True,proposed_preprocessing= True,proposed_alpha=True,test_something = True)
    y_pred = toa.predict(X_test, w, b, a, M)
    return y_pred

In [8]:
# ####################################### TEST SIZE SCRIPT ################################
# M=10
# C=10000
# theta = 0.2
# N = 5
# test_size = [0.3]
# new_rate = [1/5]

# dataset = Co_Author

# time = datetime.now().strftime("%d%m%Y_%H%M%S")
# filename = (str(dataset).split("\\")[-1]).split(".")[0]
# filepath = f'./Experiment/Data_{filename}_TestSize.csv'
# for n in range(0,N):
#     header = ['Test Size','IR','Method', 'SP', 'SE', 'Gmean', 'F1 Score','Precision','Accuracy','AUC','Ma tran nham lan']
#     data = []
#     print("Lan boc: ", n+1)
#     for testsize in test_size:
#         for newrate in new_rate:
#             X_train, y_train, X_test, y_test = dataset.load_data(test_size=testsize, new_rate=newrate)
#             with open(f'./Experiment/Data_{filename}_{time}_TestSize.csv', 'a', encoding='UTF8', newline='') as f1:
#                 writer = csv.writer(f1)
#                 writer.writerow(header)

#                 print("ADA_Decision Tree starting...\n")
#                 y_pred = ada_decisiontree(X_train, y_train, X_test)
#                 sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#                 name = 'ADA_DSTree'
#                 data.append([testsize,newrate,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#                 print("ADA_WSVM starting...\n")
#                 y_pred = ada_wsvm(M, C, theta, X_train, y_train,X_test)
#                 sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#                 name = 'ADA_WSVM'
#                 data.append([testsize,newrate,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#                 print("ImADA1_WSVM starting...\n")
#                 y_pred = imada1_wsvm(M, C, theta, X_train, y_train,X_test)
#                 sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#                 name = 'ImADA1_WSVM'
#                 data.append([testsize,newrate,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#                 print("ImADA2_WSVM starting...\n")
#                 y_pred = imada2_wsvm(M, C, X_train, y_train,X_test)
#                 sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#                 name = 'ImADA2_WSVM'
#                 data.append([testsize,newrate,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#                 print("ImADA_12_WSVM starting...\n")
#                 y_pred = imada_12_wsvm(M, C, X_train, y_train,X_test)
#                 sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#                 name = 'ImADA_12_WSVM'
#                 data.append([testsize,newrate,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])
                
#                 writer.writerows(data)


In [11]:
######################## K-FOLD SCRIPT WITHOUT CHANGE RATE #########################

M=100
C=10000
theta = 2
N = 5
from data import Ecoli_Kfold, Haberman_KFold, Transfution_Kfold
dataset = Transfution_Kfold


time = datetime.now().strftime("%d%m%Y_%H%M%S")
filename = (str(dataset).split("\\")[-1]).split(".")[0]
filepath = f'./Experiment/Data_{filename}_Full.csv'

for n in range(0,N):
    print("Lan boc: ",n+1)
    X, y = dataset.load_data()
    print(X.shape)
    kfold_validation = StratifiedKFold(n_splits=5, shuffle=True)
    header = ['Time','Fold','Method', 'SP', 'SE', 'Gmean', 'F1 Score','Precision','Accuracy','AUC','Ma tran nham lan']
    data = []
    with open(f'./Experiment/Data_{filename}_{time}_KFold.csv', 'a', encoding='UTF8', newline='') as f1:
        writer = csv.writer(f1)
        writer.writerow(header)
        fold = 1
        for train_index, test_index in kfold_validation.split(X,y):   
            print("\nFold thu ",fold)            
            # X_train, y_train = X.iloc[train_index], y.iloc[train_index]
            # X_test, y_test = X.iloc[test_index], y.iloc[test_index]
            X_train, y_train = X[train_index], y[train_index]
            X_test, y_test = X[test_index], y[test_index]
            print(X_test.shape)
            
            #Scalling Data
            sc_X = StandardScaler()
            X_train = sc_X.fit_transform(X_train)
            X_test = sc_X.transform(X_test)
            y_train = np.array(y_train)

            print("ADA_Decision Tree starting...\n")
            y_pred = ada_decisiontree(X_train, y_train, X_test)
            sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
            name = 'ADA_DSTree'
            data.append([n+1,fold,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

            print("ADA_WSVM starting...\n")
            y_pred = ada_wsvm(M, C, theta, X_train, y_train,X_test)
            sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
            name = 'ADA_WSVM'
            data.append([n+1,fold,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

            print("ImADA1_WSVM starting...\n")
            y_pred = imada1_wsvm(M, C, theta, X_train, y_train,X_test)
            sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
            name = 'ImADA1_WSVM'
            data.append([n+1,fold,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

            print("ImADA2_WSVM starting...\n")
            y_pred = imada2_wsvm(M, C, X_train, y_train,X_test)
            sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
            name = 'ImADA2_WSVM'
            data.append([n+1,fold,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

            print("ImADA_12_WSVM starting...\n")
            y_pred = imada_12_wsvm(M, C, X_train, y_train,X_test)
            sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
            name = 'ImADA_12_WSVM'
            data.append([n+1,fold,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])
                
            fold = fold + 1
        writer.writerows(data)


Lan boc:  1
(748, 4)

Fold thu  1
(150, 4)
ADA_Decision Tree starting...

ADA_WSVM starting...



d:\APP\envs\FSVM-CIL\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ImADA1_WSVM starting...

ImADA2_WSVM starting...



d:\APP\envs\FSVM-CIL\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ImADA_12_WSVM starting...


Fold thu  2
(150, 4)
ADA_Decision Tree starting...

ADA_WSVM starting...

ImADA1_WSVM starting...

ImADA2_WSVM starting...

ImADA_12_WSVM starting...


Fold thu  3
(150, 4)
ADA_Decision Tree starting...

ADA_WSVM starting...



d:\APP\envs\FSVM-CIL\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ImADA1_WSVM starting...

ImADA2_WSVM starting...



d:\APP\envs\FSVM-CIL\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ImADA_12_WSVM starting...


Fold thu  4
(149, 4)
ADA_Decision Tree starting...

ADA_WSVM starting...



KeyboardInterrupt: 

In [ ]:
# ######################## K-FOLD SCRIPT WITH CHANGE RATE #########################

# M=10
# C=10000
# theta = 0.2
# N = 5
# new_rate = [1/5,1/7,1/9]
# from data import Co_Author_KF

# dataset = Co_Author_KF

# time = datetime.now().strftime("%d%m%Y_%H%M%S")
# filename = (str(dataset).split("\\")[-1]).split(".")[0]
# filepath = f'./Experiment/Data_{filename}_Full.csv'

# for n in range(0,N):
#     print("Lan boc: ",n+1)
#     for ir in new_rate:
#         X, y = dataset.load_data(ir)
#         print("IR = ", ir)
#         print(X.shape)
#         kfold_validation = StratifiedKFold(n_splits=5, shuffle=True)
#         header = ['Time','IR','Fold','Method', 'SP', 'SE', 'Gmean', 'F1 Score','Precision','Accuracy','AUC','Ma tran nham lan']
#         data = []
#         with open(f'./Experiment/Data_{filename}_{time}_KFold.csv', 'a', encoding='UTF8', newline='') as f1:
#             writer = csv.writer(f1)
#             writer.writerow(header)
#             fold = 1
#             for train_index, test_index in kfold_validation.split(X,y):   
#                 print("\nFold thu ",fold)            
#                 # X_train, y_train = X.iloc[train_index], y.iloc[train_index]
#                 # X_test, y_test = X.iloc[test_index], y.iloc[test_index]
#                 X_train, y_train = X[train_index], y[train_index]
#                 X_test, y_test = X[test_index], y[test_index]
#                 print(X_test.shape)
                
#                 #Scalling Data
#                 sc_X = StandardScaler()
#                 X_train = sc_X.fit_transform(X_train)
#                 X_test = sc_X.transform(X_test)
#                 y_train = np.array(y_train)

#                 print("ADA_Decision Tree starting...\n")
#                 y_pred = ada_decisiontree(X_train, y_train, X_test)
#                 sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#                 name = 'ADA_DSTree'
#                 data.append([n+1,ir,fold,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#                 print("ADA_WSVM starting...\n")
#                 y_pred = ada_wsvm(M, C, theta, X_train, y_train,X_test)
#                 sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#                 name = 'ADA_WSVM'
#                 data.append([n+1,ir,fold,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#                 print("ImADA1_WSVM starting...\n")
#                 y_pred = imada1_wsvm(M, C, theta, X_train, y_train,X_test)
#                 sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#                 name = 'ImADA1_WSVM'
#                 data.append([n+1,ir,fold,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#                 print("ImADA2_WSVM starting...\n")
#                 y_pred = imada2_wsvm(M, C, X_train, y_train,X_test)
#                 sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#                 name = 'ImADA2_WSVM'
#                 data.append([n+1,ir,fold,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])

#                 print("ImADA_12_WSVM starting...\n")
#                 y_pred = imada_12_wsvm(M, C, X_train, y_train,X_test)
#                 sp, se, gmean, f1s, pre, acc, auc, cm_WSVM = compute_metrics(y_test, y_pred)
#                 name = 'ImADA_12_WSVM'
#                 data.append([n+1,ir,fold,name,sp, se, gmean, f1s, pre, acc, auc, str(cm_WSVM)])
                    
#                 fold = fold + 1
#             writer.writerows(data)


Lan boc:  1
IR =  0.2
(120, 7)

Fold thu  1
(24, 7)
ADA_Decision Tree starting...

ADA_WSVM starting...



d:\APP\envs\FSVM-CIL\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ImADA1_WSVM starting...

ImADA2_WSVM starting...

ImADA_12_WSVM starting...


Fold thu  2
(24, 7)
ADA_Decision Tree starting...

ADA_WSVM starting...

ImADA1_WSVM starting...

ImADA2_WSVM starting...

ImADA_12_WSVM starting...


Fold thu  3
(24, 7)
ADA_Decision Tree starting...

ADA_WSVM starting...

ImADA1_WSVM starting...

ImADA2_WSVM starting...

ImADA_12_WSVM starting...


Fold thu  4
(24, 7)
ADA_Decision Tree starting...

ADA_WSVM starting...

ImADA1_WSVM starting...

ImADA2_WSVM starting...

ImADA_12_WSVM starting...


Fold thu  5
(24, 7)
ADA_Decision Tree starting...

ADA_WSVM starting...

ImADA1_WSVM starting...

ImADA2_WSVM starting...

ImADA_12_WSVM starting...

IR =  0.14285714285714285
(114, 7)

Fold thu  1
(23, 7)
ADA_Decision Tree starting...

ADA_WSVM starting...

ImADA1_WSVM starting...

ImADA2_WSVM starting...

ImADA_12_WSVM starting...


Fold thu  2
(23, 7)
ADA_Decision Tree starting...

ADA_WSVM starting...

ImADA1_WSVM starting...

ImADA2_WSVM starting...

ImADA_12_

d:\APP\envs\FSVM-CIL\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ImADA_12_WSVM starting...


Fold thu  4
(23, 7)
ADA_Decision Tree starting...



...

ADA_WSVM starting...

ImADA1_WSVM starting...

ImADA2_WSVM starting...

ImADA_12_WSVM starting...


Fold thu  4
(22, 7)
ADA_Decision Tree starting...

ADA_WSVM starting...



d:\APP\envs\FSVM-CIL\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\APP\envs\FSVM-CIL\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ImADA1_WSVM starting...

ImADA2_WSVM starting...



d:\APP\envs\FSVM-CIL\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\APP\envs\FSVM-CIL\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ImADA_12_WSVM starting...


Fold thu  5
(22, 7)
ADA_Decision Tree starting...

ADA_WSVM starting...

ImADA1_WSVM starting...

ImADA2_WSVM starting...

ImADA_12_WSVM starting...



In [ ]:
M=10
C=10000
theta = 0.2
